# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


In [1]:
import pandas as pd # Update last version of pandas
from IPython.display import display
import numpy as np
import math
import json
import ast, re
import seaborn as sns
import numpy as np

from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

pd.set_option('display.max_columns', None)

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## Data Exploration

Fristly, It is important to understand the data provided, in order to recognise some patterns or the nature of the data. I created a function that allows us to see the null values ratio, size of the dataset and a sample.

In [2]:
def analyse_dataset_integrity(df):
    print('The size of the dataset is {}'.format(len(df)))
    print('The null ratio:', '\n\n', df.isnull().sum()/len(df), '\n\n')
    print(df.describe(), '\n\n')
    print(df.head().to_string())

In [3]:
analyse_dataset_integrity(portfolio)

The size of the dataset is 10
The null ratio: 

 reward        0.0
channels      0.0
difficulty    0.0
duration      0.0
offer_type    0.0
id            0.0
dtype: float64 


          reward  difficulty   duration
count  10.000000   10.000000  10.000000
mean    4.200000    7.700000   6.500000
std     3.583915    5.831905   2.321398
min     0.000000    0.000000   3.000000
25%     2.000000    5.000000   5.000000
50%     4.000000    8.500000   7.000000
75%     5.000000   10.000000   7.000000
max    10.000000   20.000000  10.000000 


   reward                      channels  difficulty  duration     offer_type                                id
0      10       [email, mobile, social]          10         7           bogo  ae264e3637204a6fb9bb56bc8210ddfd
1      10  [web, email, mobile, social]          10         5           bogo  4d5c57ea9a6940dd891ad53e9dbe8da0
2       0          [web, email, mobile]           0         4  informational  3f207df678b143eea3cee63160fa8bed
3       5         

In [4]:
analyse_dataset_integrity(profile)

The size of the dataset is 17000
The null ratio: 

 gender              0.127941
age                 0.000000
id                  0.000000
became_member_on    0.000000
income              0.127941
dtype: float64 


                age  became_member_on         income
count  17000.000000      1.700000e+04   14825.000000
mean      62.531412      2.016703e+07   65404.991568
std       26.738580      1.167750e+04   21598.299410
min       18.000000      2.013073e+07   30000.000000
25%       45.000000      2.016053e+07   49000.000000
50%       58.000000      2.017080e+07   64000.000000
75%       73.000000      2.017123e+07   80000.000000
max      118.000000      2.018073e+07  120000.000000 


  gender  age                                id  became_member_on    income
0   None  118  68be06ca386d4c31939f3a4f0e3dd783          20170212       NaN
1      F   55  0610b486422d4921ae7d2bf64640c50b          20170715  112000.0
2   None  118  38fe809add3b4fcf9315a9694bb96ff5          20180712       NaN
3

In [5]:
# Verify that the users are unique and no duplicated
profile['id'].is_unique

True

In [6]:
analyse_dataset_integrity(transcript)

The size of the dataset is 306534
The null ratio: 

 person    0.0
event     0.0
value     0.0
time      0.0
dtype: float64 


                time
count  306534.000000
mean      366.382940
std       200.326314
min         0.000000
25%       186.000000
50%       408.000000
75%       528.000000
max       714.000000 


                             person           event                                             value  time
0  78afa995795e4d85b5d9ceeca43f5fef  offer received  {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}     0
1  a03223e636434f42ac4c3df47e8bac43  offer received  {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}     0
2  e2127556f4f64592b11af22de27a7932  offer received  {'offer id': '2906b810c7d4411798c6938adc9daaa5'}     0
3  8ec6ce2a7e7949b1bf142def7d0e0586  offer received  {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}     0
4  68617ca6246f4fbc85e91a2a49552598  offer received  {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}     0


**Observations**
* The data completeness seems to be adequate for the transcript and portfolio datasets.(Non Null Values) 
* The dataset profile, has some expected empty values (gender and income), This is expected since some people may prefer not to mention their salaries or age, in order to protect their privacy.
* The dataset portfolio has 10 offers.
* The age in the profile dataset seems biased, I will consider only people below 80 years old.
* Users in profiles are unique

## Portfolio Dataset Transformation

The protfolio dataset contains two columns **channels** and **offer_type**, which can be transformed into hot encodings.

In [7]:
def clean_string_list(text):
    return str(text).replace('[','').replace(']','').replace(' ','')

def transform_portfolio(df_portfolio):
    """
    The function makes the needed hot encodings for the portfolio dataset
    """
    # Enconde the column channels
    df_portfolio['channels'] = df_portfolio['channels'].apply(lambda row: clean_string_list(row))
    dummy_channels = df_portfolio['channels'].str.get_dummies(sep=",")
    df_portfolio = pd.concat([df_portfolio, dummy_channels], axis=1)
    # Encode the column offer_type
    df_portfolio_encoded = pd.get_dummies(df_portfolio, columns=['offer_type'])
    df_portfolio_encoded.drop(['channels'], axis=1, inplace=True)
    return df_portfolio_encoded

df_portfolio_encoded = transform_portfolio(portfolio)

In [8]:
# The portfolio ready and encoded.
df_portfolio_encoded

,reward,difficulty,duration,id,'email','mobile','social','web',offer_type_bogo,offer_type_discount,offer_type_informational
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0,1,0,0
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,1,0,0
2,0,0,4,3f207df678b143eea3cee63160fa8bed,1,1,0,1,0,0,1
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1,1,0,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1,0,1,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1,0,1,0
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1,0,1,0
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,1,1,1,0,0,0,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1,1,0,0
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,1,1,0,1,0,1,0


## Transcript Dataset Transformation

The transcript dataset will be transformed as well and as it was mentioned in the description there are 4 events and different values. We need to hot-encode those ones as well and also the time in hours to days, in order to make it more convenient. Promotions in days duration are more likely to be understood by a human than hours.

In [9]:
transcript.groupby(['event']).agg({'value': 'first', 'time': 'last'})

,value,time
event,,
offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,714
offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},576
offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},714
transaction,{'amount': 0.8300000000000001},714


In order to create a clean **transcript** dataframe, all the needed manipulations to extract values and convert them will be included in a function, which performs all the changes needed. In addition, **We will need to split the offers and transactions in two datasets based on events, in order to later relate them with the users.**

In [23]:
def transform_transcript(df_transcript):
    """
    The function transform the transcript dataframe with hot-encodings and returns separated dataframes for all
    transcripts, the sum of transactions grouped by user and the offers.
    """
        
    # Create a new column with time in days
    df_transcript['time_days'] = df_transcript['time']/24
    
    # Replace some tricky blanc spaces
    df_transcript['value'] = df_transcript['value'].apply(lambda x: str(x).replace('offer id', 'offer_id'))
    # Convert all the values in value column in dictionaries
    df_transcript['value'] = df_transcript['value'].apply(lambda x: ast.literal_eval(x))
    
    df_transcript['offer_received'] = df_transcript['event'].apply(lambda x: 1 if x == 'offer received' else 0)
    df_transcript['offer_completed'] = df_transcript['event'].apply(lambda x: 1 if x == 'offer completed' else 0)
    df_transcript['offer_viewed'] = df_transcript['event'].apply(lambda x: 1 if x == 'offer viewed' else 0)
    
    df_transactions = df_transcript.loc[df_transcript['event'].isin(['transaction'])].copy().reset_index(drop=True)
    df_offers = df_transcript.loc[df_transcript['event'].isin(['offer completed','offer received','offer viewed'])].copy().reset_index(drop=True)
    
    df_transactions['amount'] = df_transactions['value'].apply(lambda x: float(x['amount']))
    df_offers['offer_id'] = df_offers['value'].apply(lambda x: x['offer_id'])    
    
    # Transactions is a one many relation. One user could have several transactions
    df_transactions_grouped = df_transactions.groupby(['person']).agg({'time': sum,
                                                                       'time_days': sum,
                                                                       'amount': sum}).sort_values(by='amount', ascending=False).reset_index(level=0)
    df_offers.drop(['value'], axis=1, inplace=True)         
    return df_transcript, df_transactions_grouped, df_offers
            
df_transcript_mix_encoded, df_transactions, df_offers = transform_transcript(transcript)

In [24]:
df_transcript_mix_encoded

,person,event,value,time,time_days,offer_received,offer_completed,offer_viewed
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0.00,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0.00,1,0,0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer_id': '2906b810c7d4411798c6938adc9daaa5'},0,0.00,1,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0.00,1,0,0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0.00,1,0,0
...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714,29.75,0,0,0
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714,29.75,0,0,0
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714,29.75,0,0,0
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714,29.75,0,0,0


In [25]:
# Sum of the transactions per user
df_transactions.head(5)

,person,time,time_days,amount
0,3c8d541112a74af99e88abbd0692f00e,3990,166.25,1608.69
1,f1d65ae63f174b8f80fa063adcaa63b7,4788,199.50,1365.66
2,ae6f43089b674728a50b8727252d3305,5556,231.50,1327.74
3,626df8678e2a4953b9098246418c9cfa,4260,177.50,1321.42
4,73afdeca19e349b98f09e928644610f8,5004,208.50,1319.97


In [26]:
df_transactions.describe()

,time,time_days,amount
count,16578.000000,16578.000000,16578.000000
mean,3198.352515,133.264688,107.096874
std,1955.978490,81.499104,126.393939
min,12.000000,0.500000,0.050000
25%,1710.000000,71.250000,23.682500
50%,2778.000000,115.750000,72.410000
75%,4296.000000,179.000000,150.937500
max,15036.000000,626.500000,1608.690000


In [27]:
df_offers.head(5)

,person,event,time,time_days,offer_received,offer_completed,offer_viewed,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,0.0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0.0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,0,0.0,1,0,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,0.0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,0.0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0


In [28]:
df_offers.groupby(['person','offer_id','offer_received']).sum()

time  \
person                           offer_id                         offer_received         
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5 0                576   
                                                                  1                576   
                                 3f207df678b143eea3cee63160fa8bed 0                372   
                                                                  1                336   
                                 5a8bc65990b245e5a138643cd4eb9837 0                192   
...                                                                                ...   
ffff82501cea40309d5fdd7edcca4a07 2906b810c7d4411798c6938adc9daaa5 1               1320   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9 0               1038   
                                                                  1                504   
                                 fafdcd668e3743c1bb461111dcafc2a4 0                 66   
                                                                  1                  0   

                                                                                  time_days  \
person                           offer_id                         offer_received              
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5 0                   24.00   
                                                                  1                   24.00   
                                 3f207df678b143eea3cee63160fa8bed 0                   15.50   
                                                                  1                   14.00   
                                 5a8bc65990b245e5a138643cd4eb9837 0                    8.00   
...                                                                                     ...   
ffff82501cea40309d5fdd7edcca4a07 2906b810c7d4411798c6938adc9daaa5 1                   55.00   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9 0                   43.25   
                                                                  1                   21.00   
                                 fafdcd668e3743c1bb461111dcafc2a4 0                    2.75   
                                                                  1                    0.00   

                                                                                  offer_completed  \
person                           offer_id                         offer_received                    
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5 0                             1   
                                                                  1                             0   
                                 3f207df678b143eea3cee63160fa8bed 0                             0   
                                                                  1                             0   
                                 5a8bc65990b245e5a138643cd4eb9837 0                             0   
...                                                                                           ...   
ffff82501cea40309d5fdd7edcca4a07 2906b810c7d4411798c6938adc9daaa5 1                             0   
                                 9b98b8c7a33c4b65b9aebfe6a799e6d9 0                             1   
                                                                  1                             0   
                                 fafdcd668e3743c1bb461111dcafc2a4 0                             1   
                                                                  1                             0   

                                                                                  offer_viewed  
person                           offer_id                         offer_received                
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5 0                          0  
                                                                  1   

#### Important remarks

* Transcript has all the records of acivity of the user by purchasing products and the offers sent to the users. By grouping and sum the expenses of users, we se that some of them spend a significant amount of money with a mas 1608 dollars in a period of two years. 


* The columns related with time here may be deleted since only represent the intrerval of time, in which the users were monitored for this source dataset. It just serves as a reference. However, they will not be a direct feature to consider for a predictor or a recommendation in a first place.

## Profile Dataset Transformation

Here the profile dataset will be transformed, cleaned. I would start with the coherence of the data in some columns such as **age** and **gender**.

In [16]:
profile.groupby(['age']).count()

,gender,id,became_member_on,income
age,,,,
18,70,70,70,70
19,135,135,135,135
20,135,135,135,135
21,140,140,140,140
22,131,131,131,131
...,...,...,...,...
98,5,5,5,5
99,5,5,5,5
100,12,12,12,12


The age column seems to be biased since they have customers with an age max 101. It is feasible but I would like to focus on the age segment **under 80 years old.**

In [17]:
profile.groupby(['gender']).count()

,age,id,became_member_on,income
gender,,,,
F,6129,6129,6129,6129
M,8484,8484,8484,8484
O,212,212,212,212


In the gender case I detected three types of gender, and I assume that **O** is an special group that I will not consider in this analysis, since the proportion of them compared with the entire population is not significant.

**Only Masculine and Femenine will be hot encoded to 1 and 0 respecttively.**

We need to create a function to hot encode the profile dataset.

In [29]:
def transform_profile(df_profile):
    
    df_profile = df_profile[df_profile['age'] < 81]
    df_profile = df_profile.drop(df_profile[df_profile['age'].isnull()].index)
    df_profile = df_profile.drop(df_profile[df_profile['gender'].isnull()].index)
    df_profile['gender_map'] = df_profile['gender'].apply(lambda x: 1 if x == 'M' else 0)
    
    df_profile['became_member_on'] = df_profile['became_member_on'].astype('str')
    df_profile['became_member_on'] = df_profile['became_member_on'].astype('datetime64[ns]')
    df_profile['start_year'] = df_profile['became_member_on'].dt.year
    df_profile["start_month"] = df_profile['became_member_on'].dt.month
    
    return df_profile

df_profile_encoded = transform_profile(profile)
df_profile_encoded

,gender,age,id,became_member_on,income,gender_map,start_year,start_month
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,0,2017,7
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,0,2017,5
5,M,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,1,2018,4
8,M,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0,1,2018,2
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0,1,2017,11
...,...,...,...,...,...,...,...,...
16993,M,60,cb23b66c56f64b109d673d5e56574529,2018-05-05,113000.0,1,2018,5
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,2018-06-04,54000.0,0,2018,6
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,2018-07-13,72000.0,1,2018,7
16997,M,49,01d26f638c274aa0b965d24cefe3183f,2017-01-26,73000.0,1,2017,1


In [30]:
# Verify again that the user values in profile are unique
df_profile_encoded['id'].is_unique

True

In [31]:
# Verify again that age is under 80 years old
df_profile_encoded.describe()

,age,income,gender_map,start_year,start_month
count,13839.000000,13839.000000,13839.000000,13839.000000,13839.000000
mean,52.088157,64950.285425,0.583062,2016.621143,6.696149
std,15.562569,21474.598708,0.493070,1.205106,3.489183
min,18.000000,30000.000000,0.000000,2013.000000,1.000000
25%,41.000000,48000.000000,0.000000,2016.000000,4.000000
50%,54.000000,63000.000000,1.000000,2017.000000,7.000000
75%,64.000000,79000.000000,1.000000,2017.000000,10.000000
max,80.000000,120000.000000,1.000000,2018.000000,12.000000


### Combining all datasets - Big Table

The time has arrive to combine the data in a way to get the full picture of the entire activity and try to detect some patterns. After analysing all the datasets, I consider that my main base dataset is the offers because one user can receive several offers and it is not proper to reduce one user per offer. 


The merging strategy will be based on joins as in SQL and I will do the following sequential joins to achieve the big table result with all the features.


**df_offers <-- df_portfolio_encoded <-- df_profile_encoded <--df_transactions == big features table**

In [21]:
def combining_datasets(df_profile, df_portfolio, df_transcript):
    
    
    return data

## Data Analysis

In this section

In [22]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Starbucks_Capstone_notebook.ipynb'])

0